In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# My imports
import pandas_profiling
import matplotlib.pyplot as plt
import sklearn

In [ ]:
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
shops.head()

In [ ]:
items.head()

In [ ]:
item_categories.head()

In [ ]:
sales_train.head()

In [ ]:
joined = sales_train.merge(shops, left_on='shop_id', right_on='shop_id')
joined = joined.merge(items, left_on='item_id', right_on='item_id')
joined = joined.merge(item_categories, left_on='item_category_id', right_on='item_category_id')
joined.head()

In [ ]:
print(sales_train.shape)
print(joined.shape)

In [ ]:
joined.groupby('date_block_num')['item_cnt_day'].sum().plot()

## Let's try plotting the time series for each shop-item combination

In [ ]:
joined['uid'] = joined['shop_id'].map(str).str.cat(joined['item_id'].map(str))
grouped = joined.groupby(['date_block_num', 'uid'])['item_cnt_day'].sum().unstack()
grouped.fillna(value=0, inplace=True)
grouped.head()

## Let's try plotting the time series for each shop

In [ ]:
grouped = joined.groupby(['date_block_num', 'shop_id'])['item_cnt_day'].sum().unstack()
grouped.fillna(value=0, inplace=True)
grouped.plot()

In [ ]:
joined.groupby('shop_id')['item_cnt_day'].sum().hist(bins=200)

## Let's try plotting the time series for each item

In [ ]:
grouped = joined.groupby(['date_block_num', 'item_id'])['item_cnt_day'].sum().unstack()
grouped.fillna(value=0, inplace=True)
grouped[grouped.columns[:100]].plot()

In [ ]:
grouped[grouped.columns[500:600]].plot()

In [ ]:
joined.groupby('item_id')['item_cnt_day'].sum().hist(bins=100, range=[0,500])

## Let's fit linear regression models to model monthly drift
### 1. Overall dataset

In [ ]:
linear_model = sklearn.linear_model.ridg